# This is the notebook for my capstone project in Coursera Data Science

In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

## Week 3: Segmenting and Clustering Neighborhoods in Toronto

### Download data

In [2]:
DF_list=pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
Toronto_DF=DF_list[0]

### Drop rows with no assigned borough

In [3]:
Toronto_DF = Toronto_DF[Toronto_DF.Borough != 'Not assigned']
Toronto_DF.shape

(103, 3)

### Retrieve Longitude and Latitude

In [4]:
DF_coord=pd.read_csv('http://cocl.us/Geospatial_data')
DF_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
Toronto_DF=pd.merge(Toronto_DF,DF_coord, on ='Postal Code')
Toronto_DF.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Now that I have the coordinates with the Neighborhoods, I can start exploring

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [7]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="cn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Create a map of Toronto with the Neighborhoods superimposed

In [8]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_DF['Latitude'], Toronto_DF['Longitude'], Toronto_DF['Borough'], Toronto_DF['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### I'm going to limit the anaysis to only Neighborhoods whose name contains 'P'

In [9]:
TorontoTrimmed_DF = Toronto_DF[Toronto_DF['Neighborhood'].str.contains('P')]
TorontoTrimmed_DF.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724


In [10]:
TorontoTrimmed_DF.shape

(30, 5)

In [11]:
# create a trimmed map of Toronto using latitude and longitude values
map_TorontoTrimmed = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(TorontoTrimmed_DF['Latitude'], TorontoTrimmed_DF['Longitude'], TorontoTrimmed_DF['Borough'], TorontoTrimmed_DF['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TorontoTrimmed)  
    
map_TorontoTrimmed

## Defining Foursquare creditials

In [12]:
CLIENT_ID = '3WIDPJJAES1OEFHQIK5DXPRDL4UPK2PIR2HUFKGTLHARJIAD' # your Foursquare ID
CLIENT_SECRET = 'QY4ZZLM4S2KGI5ZIWZ5IIXY51AS02KKUWGO0VILMNV4DSNT0' # your Foursquare Secret
VERSION = '20200625'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3WIDPJJAES1OEFHQIK5DXPRDL4UPK2PIR2HUFKGTLHARJIAD
CLIENT_SECRET:QY4ZZLM4S2KGI5ZIWZ5IIXY51AS02KKUWGO0VILMNV4DSNT0


## Create a function to explore the venues near each neighborhood

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Run the code

In [14]:
Toronto_venues = getNearbyVenues(names=TorontoTrimmed_DF['Neighborhood'],
                                   latitudes=TorontoTrimmed_DF['Latitude'],
                                   longitudes=TorontoTrimmed_DF['Longitude']
                                  )

Parkwoods
Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Parkview Hill, Woodbine Gardens
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Berczy Park
Thorncliffe Park
Northwood Park, York University
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Brockton, Parkdale Village, Exhibition Place
North Park, Maple Leaf Park, Upwood Park
Bedford Park, Lawrence Manor East
Lawrence Park
Dorset Park, Wexford Heights, Scarborough Town Centre
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
Parkdale, Roncesvalles
Canada Post Gateway Processing Centre
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Stn A PO Boxes
First Canadian Place, Underground city
Business reply mail Proce

In [15]:
print(Toronto_venues.shape)
Toronto_venues.head()

(674, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
3,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
4,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


### How many venues are near each Neighborhood?

In [16]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
Canada Post Gateway Processing Centre,13,13,13,13,13,13
"Dorset Park, Wexford Heights, Scarborough Town Centre",7,7,7,7,7,7
"First Canadian Place, Underground city",100,100,100,100,100,100
"Forest Hill North & West, Forest Hill Road Park",4,4,4,4,4,4
"High Park, The Junction South",25,25,25,25,25,25


### How many unique categories are there?

In [17]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 181 uniques categories.


# Analyzing each Neighborhood

In [18]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# Create function to reorder columns
def reorder_columns(columns, first_cols=[]):
    columns=list(set(columns)-set(first_cols))
    new_order=first_cols+columns
    return new_order

# move Neighborhood to first column
my_list=Toronto_onehot.columns.tolist()
reordered_cols=reorder_columns(my_list, first_cols=['Neighborhood'])
Toronto_onehot=Toronto_onehot[reordered_cols]

Toronto_onehot.head()

,Neighborhood,Fried Chicken Joint,Bus Line,American Restaurant,Cajun / Creole Restaurant,Opera House,Stadium,Comic Shop,Massage Studio,Theater,Liquor Store,Mexican Restaurant,Beer Bar,Health Food Store,Tea Room,Athletics & Sports,Hardware Store,Intersection,Beer Store,Food Truck,Art Gallery,Garden,Gastropub,Pub,Swim School,Malay Restaurant,Performing Arts Venue,Record Shop,Department Store,Metro Station,Poutine Place,Optical Shop,Shopping Mall,Brazilian Restaurant,Flea Market,Monument / Landmark,Men's Store,Eastern European Restaurant,Belgian Restaurant,Ice Cream Shop,Deli / Bodega,Korean Restaurant,Vegetarian / Vegan Restaurant,Breakfast Spot,Gluten-free Restaurant,Kids Store,Fast Food Restaurant,Gaming Cafe,Filipino Restaurant,Cupcake Shop,Trail,Mac & Cheese Joint,Food & Drink Shop,Burger Joint,Antique Shop,Café,Asian Restaurant,Basketball Stadium,Movie Theater,Burrito Place,Dog Run,Shoe Store,Bed & Breakfast,Vietnamese Restaurant,Baseball Field,Butcher,Tennis Court,Juice Bar,Caribbean Restaurant,Building,Gift Shop,Thai Restaurant,Sushi Restaurant,Clothing Store,Lounge,Farmers Market,Bank,Grocery Store,Comfort Food Restaurant,Social Club,Garden Center,Sandwich Place,Toy / Game Store,Electronics Store,Museum,Golf Course,Park,Chocolate Shop,Nightclub,Climbing Gym,Restaurant,Tailor Shop,Bar,Coffee Shop,French Restaurant,Middle Eastern Restaurant,Pharmacy,Pet Store,Hobby Shop,Gas Station,Greek Restaurant,Steakhouse,Pizza Place,Light Rail Station,Organic Grocery,Diner,Tanning Salon,Gym / Fitness Center,Seafood Restaurant,Cheese Shop,Flower Shop,Italian Restaurant,Event Space,Beach,Skate Park,Fish Market,Poke Place,Convenience Store,Miscellaneous Shop,Bus Station,Mobile Phone Shop,Smoothie Shop,Rental Car Location,Mediterranean Restaurant,Brewery,Molecular Gastronomy Restaurant,Food Court,Yoga Studio,Jewelry Store,Sports Bar,Discount Store,Bakery,Wine Shop,Fountain,Furniture / Home Store,New American Restaurant,Salon / Barbershop,Spa,Construction & Landscaping,Cosmetics Shop,Colombian Restaurant,Bookstore,Auto Workshop,Church,Irish Pub,Indian Restaurant,Creperie,BBQ Joint,Doner Restaurant,Arts & Crafts Store,Bistro,Gym,Soup Place,Speakeasy,Supplement Shop,Cocktail Bar,Cuban Restaurant,Train Station,General Travel,Hospital,Jazz Club,Distribution Center,Sporting Goods Shop,Concert Hall,Gourmet Shop,Photography Studio,Plaza,Warehouse Store,Dessert Shop,Music Venue,Japanese Restaurant,Wine Bar,Historic Site,Thrift / Vintage Store,Supermarket,Bagel Shop,College Auditorium,Hotel,Wings Joint,Salad Place,Falafel Restaurant,Chinese Restaurant
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [19]:
Toronto_onehot.shape

(674, 182)

### Grouping by Neighborhood

In [20]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Fried Chicken Joint,Bus Line,American Restaurant,Cajun / Creole Restaurant,Opera House,Stadium,Comic Shop,Massage Studio,Theater,Liquor Store,Mexican Restaurant,Beer Bar,Health Food Store,Tea Room,Athletics & Sports,Hardware Store,Intersection,Beer Store,Food Truck,Art Gallery,Garden,Gastropub,Pub,Swim School,Malay Restaurant,Performing Arts Venue,Record Shop,Department Store,Metro Station,Poutine Place,Optical Shop,Shopping Mall,Brazilian Restaurant,Flea Market,Monument / Landmark,Men's Store,Eastern European Restaurant,Belgian Restaurant,Ice Cream Shop,Deli / Bodega,Korean Restaurant,Vegetarian / Vegan Restaurant,Breakfast Spot,Gluten-free Restaurant,Kids Store,Fast Food Restaurant,Gaming Cafe,Filipino Restaurant,Cupcake Shop,Trail,Mac & Cheese Joint,Food & Drink Shop,Burger Joint,Antique Shop,Café,Asian Restaurant,Basketball Stadium,Movie Theater,Burrito Place,Dog Run,Shoe Store,Bed & Breakfast,Vietnamese Restaurant,Baseball Field,Butcher,Tennis Court,Juice Bar,Caribbean Restaurant,Building,Gift Shop,Thai Restaurant,Sushi Restaurant,Clothing Store,Lounge,Farmers Market,Bank,Grocery Store,Comfort Food Restaurant,Social Club,Garden Center,Sandwich Place,Toy / Game Store,Electronics Store,Museum,Golf Course,Park,Chocolate Shop,Nightclub,Climbing Gym,Restaurant,Tailor Shop,Bar,Coffee Shop,French Restaurant,Middle Eastern Restaurant,Pharmacy,Pet Store,Hobby Shop,Gas Station,Greek Restaurant,Steakhouse,Pizza Place,Light Rail Station,Organic Grocery,Diner,Tanning Salon,Gym / Fitness Center,Seafood Restaurant,Cheese Shop,Flower Shop,Italian Restaurant,Event Space,Beach,Skate Park,Fish Market,Poke Place,Convenience Store,Miscellaneous Shop,Bus Station,Mobile Phone Shop,Smoothie Shop,Rental Car Location,Mediterranean Restaurant,Brewery,Molecular Gastronomy Restaurant,Food Court,Yoga Studio,Jewelry Store,Sports Bar,Discount Store,Bakery,Wine Shop,Fountain,Furniture / Home Store,New American Restaurant,Salon / Barbershop,Spa,Construction & Landscaping,Cosmetics Shop,Colombian Restaurant,Bookstore,Auto Workshop,Church,Irish Pub,Indian Restaurant,Creperie,BBQ Joint,Doner Restaurant,Arts & Crafts Store,Bistro,Gym,Soup Place,Speakeasy,Supplement Shop,Cocktail Bar,Cuban Restaurant,Train Station,General Travel,Hospital,Jazz Club,Distribution Center,Sporting Goods Shop,Concert Hall,Gourmet Shop,Photography Studio,Plaza,Warehouse Store,Dessert Shop,Music Venue,Japanese Restaurant,Wine Bar,Historic Site,Thrift / Vintage Store,Supermarket,Bagel Shop,College Auditorium,Hotel,Wings Joint,Salad Place,Falafel Restaurant,Chinese Restaurant
0,"Bedford Park, Lawrence Manor East",0.000000,0.00,0.043478,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.043478,0.0,0.043478,0.000000,0.00,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,0.043478,0.043478,0.0000,0.000000,0.086957,0.043478,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.086957,0.000000,0.000000,0.086957,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.043478,0.000000,0.043478,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.086957,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000

In [21]:
Toronto_grouped.shape

(30, 182)

### What are the five most common venues in each neighborhood?

In [22]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2          Restaurant  0.09
3      Sandwich Place  0.09
4    Greek Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.03
3  Seafood Restaurant  0.03
4          Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3       Pet Store  0.05
4    Intersection  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
            venue  freq
0      Skate Park  0.07
1            Park  0.07
2         Brewery  0.07
3  Farmers Market  0.07
4          Garden  0.07


----Canada Post Gateway Processing Centre----
                       venue  freq
0                Coffee Shop  0.15
1                      Hotel  0.15
2        

### And now a dataframe in descending order for most popular venues

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bedford Park, Lawrence Manor East",Italian Restaurant,Restaurant,Coffee Shop,Sandwich Place,Thai Restaurant,Comfort Food Restaurant,Juice Bar,Indian Restaurant,Pub,Sushi Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Cheese Shop,Beer Bar,Café,Bakery,Beach,Fish Market
2,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bakery,Pet Store,Italian Restaurant,Performing Arts Venue,Convenience Store,Burrito Place,Intersection
3,"Business reply mail Processing Centre, South C...",Fast Food Restaurant,Brewery,Burrito Place,Gym / Fitness Center,Garden,Light Rail Station,Pizza Place,Skate Park,Farmers Market,Auto Workshop
4,Canada Post Gateway Processing Centre,Hotel,Coffee Shop,Fried Chicken Joint,Middle Eastern Restaurant,Gym,Burrito Place,Intersection,Mediterranean Restaurant,Sandwich Place,Gas Station


## Clustering the Neighborhoods

In [24]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 2, 1, 1])

### Adding Cluster info

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = TorontoTrimmed_DF

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Park,Food & Drink Shop,Chinese Restaurant,Basketball Stadium,Butcher,Baseball Field,Vietnamese Restaurant,Bed & Breakfast,Shoe Store,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Theater,Breakfast Spot,Café,Distribution Center,Beer Store,Art Gallery
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Diner,Sushi Restaurant,Fried Chicken Joint,Arts & Crafts Store,Gym,Burrito Place,Yoga Studio,Café,Distribution Center
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1,Pizza Place,Fast Food Restaurant,Pharmacy,Athletics & Sports,Gym / Fitness Center,Gastropub,Intersection,Pet Store,Breakfast Spot,Bank
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,0,Golf Course,Chinese Restaurant,Asian Restaurant,Butcher,Baseball Field,Vietnamese Restaurant,Bed & Breakfast,Shoe Store,Dog Run,Burrito Place


In [26]:
Toronto_merged.dropna(inplace=True)
Toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Park,Food & Drink Shop,Chinese Restaurant,Basketball Stadium,Butcher,Baseball Field,Vietnamese Restaurant,Bed & Breakfast,Shoe Store,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Theater,Breakfast Spot,Café,Distribution Center,Beer Store,Art Gallery
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Diner,Sushi Restaurant,Fried Chicken Joint,Arts & Crafts Store,Gym,Burrito Place,Yoga Studio,Café,Distribution Center
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1,Pizza Place,Fast Food Restaurant,Pharmacy,Athletics & Sports,Gym / Fitness Center,Gastropub,Intersection,Pet Store,Breakfast Spot,Bank
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,0,Golf Course,Chinese Restaurant,Asian Restaurant,Butcher,Baseball Field,Vietnamese Restaurant,Bed & Breakfast,Shoe Store,Dog Run,Burrito Place


# Visualizing it!!!!

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the Clusters

In [28]:
# Cluster 1 (cluster label = 0)
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"West Deane Park, Princess Gardens, Martin Grov...",0,Golf Course,Chinese Restaurant,Asian Restaurant,Butcher,Baseball Field,Vietnamese Restaurant,Bed & Breakfast,Shoe Store,Dog Run,Burrito Place


In [29]:
# Cluster 2 (cluster label = 1)
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Regent Park, Harbourfront",1,Coffee Shop,Bakery,Park,Pub,Theater,Breakfast Spot,Café,Distribution Center,Beer Store,Art Gallery
4,"Queen's Park, Ontario Provincial Government",1,Coffee Shop,Diner,Sushi Restaurant,Fried Chicken Joint,Arts & Crafts Store,Gym,Burrito Place,Yoga Studio,Café,Distribution Center
8,"Parkview Hill, Woodbine Gardens",1,Pizza Place,Fast Food Restaurant,Pharmacy,Athletics & Sports,Gym / Fitness Center,Gastropub,Intersection,Pet Store,Breakfast Spot,Bank
20,Berczy Park,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Cheese Shop,Beer Bar,Café,Bakery,Beach,Fish Market
29,Thorncliffe Park,1,Indian Restaurant,Park,Grocery Store,Yoga Studio,Burger Joint,Gym,Discount Store,Pizza Place,Gas Station,Bank
34,"Northwood Park, York University",1,Falafel Restaurant,Bar,Coffee Shop,Caribbean Restaurant,Furniture / Home Store,Metro Station,Massage Studio,Chinese Restaurant,Basketball Stadium,Vietnamese Restaurant
37,"Little Portugal, Trinity",1,Bar,Men's Store,Café,Asian Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Vietnamese Restaurant,Malay Restaurant,Pizza Place,Mac & Cheese Joint
38,"Kennedy Park, Ionview, East Birchmount Park",1,Chinese Restaurant,Coffee Shop,Department Store,Bus Station,Hobby Shop,Movie Theater,Butcher,Baseball Field,Vietnamese Restaurant,Bed & Breakfast
43,"Brockton, Parkdale Village, Exhibition Place",1,Café,Breakfast Spot,Coffee Shop,Bakery,Pet Store,Italian Restaurant,Performing Arts Venue,Convenience Store,Burrito Place,Intersection
55,"Bedford Park, Lawrence Manor East",1,Italian Restaurant,Restaurant,Coffee Shop,Sandwich Place,Thai Restaurant,Comfort Food Restaurant,Juice Bar,Indian Restaurant,Pub,Sushi Restaurant


In [30]:
# Cluster 3 (cluster label = 2)
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,2,Park,Food & Drink Shop,Chinese Restaurant,Basketball Stadium,Butcher,Baseball Field,Vietnamese Restaurant,Bed & Breakfast,Shoe Store,Dog Run
49,"North Park, Maple Leaf Park, Upwood Park",2,Park,Construction & Landscaping,Bakery,Trail,Chinese Restaurant,Asian Restaurant,Vietnamese Restaurant,Bed & Breakfast,Shoe Store,Dog Run
68,"Forest Hill North & West, Forest Hill Road Park",2,Park,Trail,Sushi Restaurant,Jewelry Store,Chinese Restaurant,Basketball Stadium,Baseball Field,Vietnamese Restaurant,Bed & Breakfast,Shoe Store
83,"Moore Park, Summerhill East",2,Tennis Court,Park,Asian Restaurant,Butcher,Baseball Field,Vietnamese Restaurant,Bed & Breakfast,Shoe Store,Dog Run,Burrito Place


In [31]:
# Cluster 4 (cluster label = 3)
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"Rouge Hill, Port Union, Highland Creek",3,Bar,Chinese Restaurant,Basketball Stadium,Tennis Court,Butcher,Baseball Field,Vietnamese Restaurant,Bed & Breakfast,Shoe Store,Dog Run


In [32]:
# Cluster 5 (cluster label = 4)
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
101,"Old Mill South, King's Mill Park, Sunnylea, Hu...",4,Baseball Field,Chinese Restaurant,Juice Bar,Butcher,Vietnamese Restaurant,Bed & Breakfast,Shoe Store,Dog Run,Burrito Place,Movie Theater
